In [1]:
import os 
os.chdir('..')

import pandas as pd
import tellurium as te
from src.odbm.odbm_main import ModelBuilder


In [2]:
#Define DataFrames of species, reactions from input model
model_species = pd.read_csv('src/frenda_brenda/Files/SpeciesBaseMechanisms.csv')
model_rxns = pd.read_csv('src/frenda_brenda/Files/Reaction.csv')

# model_rxns['Id'] = model_rxns['Enzyme']+'; ' +model_rxns['Substrates']+'; '+model_rxns['Products']
# model_rxns.drop_duplicates(subset='Id', inplace=True)

In [3]:
#Generate and save model to txt file
modelfile = 'src/frenda_brenda/Files/240424MC_FULL.txt'
myModel = ModelBuilder(model_species, model_rxns)
myModel.saveModel(modelfile)

r = []
while not r:
    try:
        r  = te.loada(myModel.compile())
    except Exception as e:
        myModel.addSpecies(str(e).split("'")[1], 0.001)
        print(str(e).split("'")[1])

# s = r.simulate(0,10)

In [102]:
import numpy as np
mean = {}
fixed = myModel.compile()
    
for k in ['Km','Kcat_F','Kcat_R','Ki']:
    labels = [K for K in r.getGlobalParameterIds() if k in K]
    mean[k] = np.nanmean([r[l] for l in labels])
    for l in labels:
        if np.isnan(r[l]):
            fixed = fixed.replace(l+'=nan', l+'='+str(mean['Km']))

with open('models/240226MC_FULL_fixed.txt', 'w') as f:
    f.write(fixed)

r  = te.loada(fixed)

In [11]:
with open('models/240226MC_FULL_fixed.txt', 'r') as f:
    fixed = f.read()

r  = te.loada(fixed)

with open('models/240226MC_FULL.sbml', 'w') as f:
    # Write the data to the file
    f.write(r.getSBML())

In [15]:
with open('models/240226MC_FULL_fixed.txt', 'r') as f:
    fixed = f.read()

r  = te.loada(fixed)

all_s = np.unique(r.getFloatingSpeciesIds()+ [i.split('_')[1] for i in r.getGlobalParameterIds() if 'Ki' in i])
pre = '# Initialize species \n'
for s in all_s:
    pre += f"species {s};\n"

r2  = te.loada(pre+fixed)
with open('models/240226MC_FULL.sbml', 'w') as f:
    # Write the data to the file
    f.write(r2.getSBML())

In [ ]:
import re

with open('models/240226MC_FULL_fixed.txt', 'r') as f:
    fixed = f.read()
# Define the pattern to match "Rxx" in Kms
pattern = lambda p,x: r"\b"+p+r"_(\w+)_"+x+r"\b"

# Define the replacement string
replacement = lambda p,x: p+r"_\1_"+x

# model_rxns['EC'] = model_rxns['EC'].apply(lambda x: 'EC'+x.replace('.',''))
r_to_ec_map = {l:ec for l,ec in zip(model_rxns['Label'],model_rxns['EC'])}

for rid in model_rxns['Label'].values:
    fixed = re.sub(pattern('Km',rid), replacement('Km',r_to_ec_map[rid]), fixed)
    fixed = re.sub(pattern('Ki',rid), replacement('Ki',r_to_ec_map[rid]), fixed)
    fixed = re.sub(pattern('Gi',rid), replacement('Gi',r_to_ec_map[rid]), fixed)

r2  = te.loada(fixed)
with open('models/240226MC_FULL.sbml', 'w') as f:
    # Write the data to the file
    f.write(r2.getSBML())

In [52]:
r2  = te.loada(fixed)
with open('models/240226MC_FULL.sbml', 'w') as f:
    # Write the data to the file
    f.write(r2.getSBML())

In [55]:
len(r2.getGlobalParameterIds())

1283